# Laboratorio 6 - GPUs con Python
**alumno09:** Laura Llamas López

### Numpy Code

In [ ]:
import numpy as np

# Example: Large matrices (adjust size as needed)
n = 7000  # For very large matrices, ensure you have enough RAM
A = np.random.rand(n, n).astype(np.float32)
B = np.random.rand(n, n).astype(np.float32)

C = np.dot(A, B)  # warm-up and Matrix multiplication

%timeit -r 2 -o np.dot(A, B)

print(f"Result shape: {C.shape}")
print(f"Result type: {C.dtype}")


1.06 s ± 5.51 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Result shape: (7000, 7000)
Result type: float32


### Pytorch code con GPU

Utilizamos Pytorch para realizar la multiplicación de matrices en GPU. Pytorch está optimizado para operaciones de Deep Learning y aprovecha automáticamente la GPU cuando está disponible.

Mediremos el tiempo en dos escenarios:
1. **Con transferencia de datos**: Matrices generadas en CPU (NumPy) y copiadas a GPU
2. **Sin transferencia**: Matrices generadas directamente en GPU usando Pytorch

In [ ]:
import torch
import time

# Verificar si hay GPU disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# =============================================================================
# CASO 1: CON TRANSFERENCIA DE DATOS (NumPy -> GPU)
# =============================================================================
print("\n" + "="*70)
print("CASO 1: Con transferencia de datos (CPU -> GPU)")
print("="*70)

def matmul_pytorch_with_transfer(A_np, B_np):
    """Convierte arrays NumPy a tensores Pytorch en GPU, multiplica, y devuelve a CPU"""
    A_gpu = torch.from_numpy(A_np).to(device)  # CPU -> GPU
    B_gpu = torch.from_numpy(B_np).to(device)
    C_gpu = torch.matmul(A_gpu, B_gpu)         # Multiplicación en GPU
    C_cpu = C_gpu.cpu().numpy()                # GPU -> CPU
    return C_cpu

# Warmup
_ = matmul_pytorch_with_transfer(A, B)
if torch.cuda.is_available():
    torch.cuda.synchronize()

# Medir tiempo
times_with_transfer = []
for _ in range(5):
    start = time.time()
    C_torch_transfer = matmul_pytorch_with_transfer(A, B)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    end = time.time()
    times_with_transfer.append(end - start)

time_with_transfer = np.mean(times_with_transfer) * 1e3
print(f"Tiempo con transferencia: {time_with_transfer:.3f} ms")
print(f"Result shape: {C_torch_transfer.shape}")
print(f"Result type: {C_torch_transfer.dtype}")

# =============================================================================
# CASO 2: SIN TRANSFERENCIA (generación directa en GPU)
# =============================================================================
print("\n" + "="*70)
print("CASO 2: Sin transferencia (matrices creadas directamente en GPU)")
print("="*70)

# Crear matrices directamente en GPU
A_gpu = torch.rand(n, n, dtype=torch.float32, device=device)
B_gpu = torch.rand(n, n, dtype=torch.float32, device=device)

# Warmup
C_gpu = torch.matmul(A_gpu, B_gpu)
if torch.cuda.is_available():
    torch.cuda.synchronize()

# Medir tiempo
times_no_transfer = []
for _ in range(5):
    start = time.time()
    C_gpu = torch.matmul(A_gpu, B_gpu)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    end = time.time()
    times_no_transfer.append(end - start)

time_no_transfer = np.mean(times_no_transfer) * 1e3
print(f"Tiempo sin transferencia: {time_no_transfer:.3f} ms")
print(f"Result shape: {C_gpu.shape}")
print(f"Result type: {C_gpu.dtype}")

# =============================================================================
# RESUMEN
# =============================================================================
print("\n" + "="*70)
print("RESUMEN - Pytorch con GPU")
print("="*70)
print(f"Con transferencia de datos: {time_with_transfer:.3f} ms")
print(f"Sin transferencia:          {time_no_transfer:.3f} ms")
if time_with_transfer > time_no_transfer:
    print(f"Overhead de transferencia:  {time_with_transfer - time_no_transfer:.3f} ms")
    print(f"Factor: {time_with_transfer / time_no_transfer:.2f}x más lento con transferencia")

## Resultados de la ejecución en bohr-gpu

### Salida de la ejecución

```
========================================
Ejecutando matrix-mult-alumno09.ipynb
========================================
580 ms ± 1.03 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Result shape: (7000, 7000)
Result type: float32
Usando dispositivo: cuda
GPU: NVIDIA GeForce RTX 2080 Ti

======================================================================
CASO 1: Con transferencia de datos (CPU -> GPU)
======================================================================
Tiempo con transferencia: 106.532 ms
Result shape: (7000, 7000)
Result type: float32

======================================================================
CASO 2: Sin transferencia (matrices creadas directamente en GPU)
======================================================================
Tiempo sin transferencia: 50.373 ms
Result shape: torch.Size([7000, 7000])
Result type: torch.float32

======================================================================
RESUMEN - Pytorch con GPU
======================================================================
Con transferencia de datos: 106.532 ms
Sin transferencia:          50.373 ms
Overhead de transferencia:  56.159 ms
Factor: 2.11x más lento con transferencia
```

### Tabla comparativa de resultados

| Método | Tiempo | Speedup vs NumPy CPU |
|--------|--------|----------------------|
| **NumPy CPU** | 580 ms | 1.0x (baseline) |
| **Pytorch GPU con transferencia** | 106.5 ms | **5.45x más rápido** |
| **Pytorch GPU sin transferencia** | 50.4 ms | **11.51x más rápido** |

### Análisis de resultados

**Nota**: Los resultados muestran `torch.Size([7000, 7000])` en lugar de la tupla `(7000, 7000)` de NumPy porque Pytorch utiliza su propio tipo para representar las dimensiones de los tensores. Ambos representan lo mismo: una matriz cuadrada de 7000×7000.

Los resultados de la multiplicación de matrices 7000×7000 en bohr demuestran la enorme potencia de Pytorch para operaciones matriciales intensivas.

**Pytorch sin transferencia** logra un speedup de 11.51x sobre NumPy CPU, reduciendo el tiempo de 580 ms a solo 50.4 ms. Este rendimiento excepcional se debe a que la multiplicación de matrices es una operación fundamental en Deep Learning y está extremadamente optimizada en Pytorch mediante el uso de bibliotecas BLAS optimizadas para GPU (cuBLAS). Además, al generar las matrices directamente en GPU con `torch.rand(..., device='cuda')`, evitamos completamente las transferencias de memoria.

**Pytorch con transferencia** sigue ofreciendo un speedup significativo de 5.45x (106.5 ms), a pesar de incluir el costo de convertir los arrays NumPy a tensores Pytorch y copiarlos a GPU. El **overhead de transferencia** es de 56.2 ms, lo cual representa un factor de 2.11x de ralentización. Este overhead incluye tres operaciones: convertir dos matrices NumPy (7000×7000 cada una, ~196 MB en total de float32) a tensores Pytorch, copiarlas a memoria GPU, y devolver el resultado a CPU. A pesar de este costo considerable, la operación completa con transferencias sigue siendo mucho más rápida que NumPy CPU.

En conclusión, Pytorch es excepcionalmente eficiente para multiplicación de matrices en GPU, especialmente cuando los datos ya residen en memoria GPU o cuando el tamaño del problema es lo suficientemente grande para que el cómputo compense el overhead de transferencia. Este rendimiento explica por qué Pytorch es la librería dominante en Deep Learning, donde las operaciones matriciales masivas son ubicuas en redes neuronales.